In [1]:
from braininvaders2015a.dataset import BrainInvaders2015a
import mne
from mne import create_info
from mne.io import RawArray
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mne import Epochs, find_events
from IPython.display import clear_output

In [2]:
dataset = BrainInvaders2015a()
data_subject = dataset._get_single_subject_data(3)
data_ses1_run1 = data_subject['session_1']['run_1']
print(data_subject)

{'session_1': {'run_1': <RawArray | 33 x 132160 (258.1 s), ~33.3 MB, data loaded>}, 'session_2': {'run_1': <RawArray | 33 x 95520 (186.6 s), ~24.1 MB, data loaded>}, 'session_3': {'run_1': <RawArray | 33 x 98464 (192.3 s), ~24.8 MB, data loaded>}}


In [3]:
def load_data(i):
    data_subject = dataset._get_single_subject_data(i)
    data_ses1_run1 = data_subject['session_1']['run_1']
    data_ses1_run1_pd = data_ses1_run1.to_data_frame()
    data_ses1_run1_pd = data_ses1_run1_pd.drop(['time'],axis = 1)
    return data_ses1_run1_pd
def filter(raw):
    raw.notch_filter(np.arange(50, 251, 50))
    raw.filter(1,24,method = 'iir')
    return raw

def df_to_raw(df):
    sfreq = 512
    ch_names = list(df.columns)
    ch_types = ['eeg'] * (len(df.columns) - 1) + ['stim']
    ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')

    df = df.T
      #mne looks at the tranpose() format
    df[:-1] *= 1e-6
      #convert from uVolts to Volts (mne assumes Volts data)

    info = create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)

    raw = mne.io.RawArray(df, info)
    raw.set_montage(ten_twenty_montage)

    #try plotting the raw data of its power spectral density
    # raw.plot_psd()

    return raw

In [4]:
def preprocess(i):
    clear_output(wait=True)
    data_subject1 = load_data(i)
    raw = df_to_raw(data_subject1)
    raw = filter(raw)
    # df = do_pca(raw)
    # raw = df_to_raw(df)
    events = find_events(raw)
    event_id = {'NonTarget': 1, 'Target': 2}
    eeg_channels = mne.pick_types(raw.info, eeg=True)
    epochs = Epochs(raw, events=events, event_id=event_id, 
                        tmin=0.0, tmax=0.7, baseline=None, preload=True,verbose=False, picks=eeg_channels)
    X = epochs.get_data()
    y = epochs.events[:, -1]
    X_new = X.reshape(X.shape[0],-1)
    return X_new,y

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
def do_pca(X):
    #scaling
    sc = StandardScaler()
    X = sc.fit_transform(X)

    # Perform PCA
    pca = PCA()
    # Determine transformed features
    X_pca = pca.fit_transform(X)
    return X_pca

In [6]:
X_train,y_train = preprocess(5)
print(X_train.shape)

Creating RawArray with float64 data, n_channels=33, n_times=150528
    Range : 0 ... 150527 =      0.000 ...   293.998 secs
Ready.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 24 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 24.00 Hz: -6.02, -6.02 dB

432 events found
Event IDs: [1 2]
(432, 11488)


GRIDSEARCH

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
X,y = preprocess(3)
X_train_pca = do_pca(X)
    #split
X_train, X_test, y_train, y_test = train_test_split(X_train_pca,
                    y, test_size=0.3, random_state=1)

Creating RawArray with float64 data, n_channels=33, n_times=132160
    Range : 0 ... 132159 =      0.000 ...   258.123 secs
Ready.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 24 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 24.00 Hz: -6.02, -6.02 dB

360 events found
Event IDs: [1 2]


In [9]:
kernel = ['linear', 'rbf']
C_range = [1, 2]
param_grid = dict(C=C_range, kernel=kernel)
grid = GridSearchCV(SVC(), param_grid=param_grid)
grid.fit(X_train, y_train)
print("=========Best parameters=======")
print(f"The best parameters are {grid.best_params_} with" +
          f"a score of {grid.best_score_:.2f}")

=========Best parameters=======
The best parameters are {'C': 1, 'kernel': 'linear'} witha score of 0.91


Model(subject1-10)

In [10]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.svm import SVC
accuracy = []
for i in range(10):
    X,y = preprocess(i+1)
    X_train_pca = do_pca(X)
    #split
    X_train, X_test, y_train, y_test = train_test_split(X_train_pca,
                    y, test_size=0.3, random_state=1)
                    
    #model
    clf = SVC(kernel='linear',C=1)
    clf.fit(X_train,y_train)
    yhat = clf.predict(X_test)
    acc = np.sum(yhat == y_test)/len(y_test)
    accuracy.append(acc)

Creating RawArray with float64 data, n_channels=33, n_times=122784
    Range : 0 ... 122783 =      0.000 ...   239.811 secs
Ready.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 24 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 24.00 Hz: -6.02, -6.02 dB

324 events found
Event IDs: [1 2]


In [11]:
for i in range(len(accuracy)):
    print(f"subject{i+1}",accuracy[i])

subject1 0.796895213454075
subject2 0.8014184397163121
subject3 0.9074074074074074
subject4 0.9538461538461539
subject5 0.9153846153846154
subject6 0.8333333333333334
subject7 0.8796296296296297
subject8 0.8769230769230769
subject9 0.9012345679012346
subject10 0.9285714285714286


from IPython.display import clear_output
num_event = []
X_train,y_train = preprocess(1)
for i in range(2,11):
    X,y = preprocess(i)
    X_train = np.vstack((X_train,X))
    y_train = np.concatenate((y_train,y),axis = 0)

    #num_event.append(X_train.shape[0])